#Gradient Descent

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/3d99d9abbb654a02b1e747bee2c029e5c0712356/7_nn_from_scratch/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']], df.bought_insurance, test_size = 0.2, random_state = 27)

In [8]:
len(X_train)

22

In [10]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [11]:
X_train_scaled

,age,affordibility
21,0.26,0
15,0.55,1
22,0.40,1
0,0.22,1
11,0.28,1
3,0.52,0
6,0.55,0
4,0.46,1
20,0.21,1
18,0.19,0


In [37]:
model = keras.Sequential([
                          keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(X_train_scaled, y_train, epochs = 1000)

Epoch 1/1000
1/1 [==============================] - 0s 380ms/step - loss: 0.7507 - accuracy: 0.4545
Epoch 2/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7503 - accuracy: 0.4545
Epoch 3/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7498 - accuracy: 0.4545
Epoch 4/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7493 - accuracy: 0.4545
Epoch 5/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7488 - accuracy: 0.4545
Epoch 6/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7484 - accuracy: 0.4545
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7479 - accuracy: 0.4545
Epoch 8/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7474 - accuracy: 0.4545
Epoch 9/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7470 - accuracy: 0.4545
Epoch 10/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7465 - accuracy: 0.4545
Epoch 11/1000
1/1

In [38]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 112ms/step - loss: 0.6033 - accuracy: 0.6667


[0.6033335328102112, 0.6666666865348816]

In [39]:
model.predict(X_test_scaled)

array([[0.59917784],
       [0.6351106 ],
       [0.39213923],
       [0.60145974],
       [0.46891758],
       [0.6195487 ]], dtype=float32)

In [40]:
coef, intercept = model.get_weights()

In [41]:
coef, intercept

(array([[0.9510262],
        [0.6691849]], dtype=float32), array([-0.695106], dtype=float32))

#Scratch

In [22]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))

In [25]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [26]:
prediction_function(0.47,1)

0.6916949164361127

Gradient Descent from scratch

In [27]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1 - epsilon) for i in y_predicted]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean( y_true*np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))

In [28]:
def sigmoid_numpy(X):
    return 1/(1+ np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [45]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs): #returns tuple of parameters
        self.w1 , self.w2, bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs)

    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age, affordibility, y_true, epochs):
        w1 = w2 = 1
        bias = 0
        rate = 0.01
        n = len(age)

        for i in range(epochs):
            weighted_sum = w1*age + w2*affordibility + bias
            y_predicted = sigmoid_numpy(weighted_sum)

            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
            w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted - y_true))

            bias_d = np.mean(y_predicted - y_true)

            w1 = w1 - rate*w1d
            w2 = w2 - rate*w2d
            bias = bias - rate * bias_d
            if i%50 == 0:
                print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        return w1, w2, bias

In [47]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs = 1000)

Epoch:0, w1:0.9994106012341517, w2:0.9986344455057053, bias:-0.0027608392224643323, loss:0.750728371337802
Epoch:50, w1:0.9731638171560307, w2:0.9363307428734264, bias:-0.13203000793265066, loss:0.7079029579451969
Epoch:100, w1:0.9529566600564018, w2:0.8853490053534824, bias:-0.24492814835533616, loss:0.6761825624463652
Epoch:150, w1:0.9382508235037219, w2:0.844806921766261, bias:-0.34298856478069817, loss:0.6530662098559269
Epoch:200, w1:0.9284141964060197, w2:0.8135694051890404, bias:-0.42796133622859656, loss:0.6363580656878232
Epoch:250, w1:0.9227923234255084, w2:0.7904074752770889, bias:-0.5016279820801846, loss:0.6242802619807626
Epoch:300, w1:0.9207581410758773, w2:0.7741132963507281, bias:-0.5656762253267942, loss:0.6154745594970022
Epoch:350, w1:0.9217406144147844, w2:0.7635697446570401, bias:-0.6216306593709651, loss:0.608945428287048
Epoch:400, w1:0.9252372562289256, w2:0.7577840789905832, bias:-0.6708249408349178, loss:0.603984287250857
Epoch:450, w1:0.9308161337980669, w2:

In [48]:
customModel.predict(X_test_scaled)

23    0.790268
9     0.816957
12    0.570951
27    0.792017
7     0.653614
25    0.805619
dtype: float64

In [49]:
model.predict(X_test_scaled)

array([[0.59917784],
       [0.6351106 ],
       [0.39213923],
       [0.60145974],
       [0.46891758],
       [0.6195487 ]], dtype=float32)